# Economics 600a Fall 2025 Prof. P. Haile Homework Assignment 1

## 1 Overview
You will estimate demand and supply in a stylized model of the market for pay-TV services. You will use a matrix programming language of your choice to create your own fake data set for the industry and do some relatively simple estimation. Then, using the **pyBLP** package of Conlon and Gortmaker, you will estimate the model and perform some merger simulations.

The pyBLP package has excellent documentation and a very helpful tutorial (which covers merger simulation), both easy to find via Google.

Please submit (on canvas) a single PDF document presenting your answers to the questions below, requested results, and well documented code. Write this up nicely, with properly formatted tables and discussion of results. You may work in groups on the coding. However, your write-ups should be your own work, and you must describe all collaboration at the beginning of your submission; this includes any use of AI.

## 2 Model
There are $T$ markets, each with four inside goods $j \in \{1,2,3,4\}$ and an outside option. Goods 1 and 2 are satellite television services (e.g., DirecTV and Dish); goods 3 and 4 are wired television services (e.g., Frontier and Comcast in New Haven). The conditional indirect utility of consumer $i$ for good $j$ in market $t$ is given by

\begin{align*}
u_{ijt} &= \beta^{(1)} x_{jt} + \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt} \quad j > 0 \\
u_{i0t} &= \epsilon_{i0t},
\end{align*}

where $x_{jt}$ is a measure of good $j$'s quality, $p_{jt}$ is its price, $satellite_{jt}$ is an indicator equal to 1 for the two satellite services, and $wired_{jt}$ is an indicator equal to 1 for the two wired services. The remaining notation is as usual in the class notes, including the i.i.d. type-1 extreme value $\epsilon_{ijt}$. Each consumer purchases the good giving them the highest conditional indirect utility.

Goods are produced by single-product firms. Firm $j$'s (log) marginal cost in market $t$ is

\begin{equation*}
\ln mc_{jt} = \gamma^{(0)} + w_{jt} \gamma^{(1)} + \omega_{jt}/8,
\end{equation*}

where $w_{jt}$ is an observed cost shifter. Firms compete by simultaneously choosing prices in each market under complete information. Firm $j$ has profit

\begin{equation*}
\sigma_{jt} = \max_{p_{jt}} (p_{jt} - mc_{jt}) s_{jt}(p_t).
\end{equation*}

## 3 Generate Fake Data

Generate a data set from the model above. Let

\begin{align*}
\beta^{(1)} &= 1, \quad \beta_i^{(k)} \sim \text{iid } N(4,1) \text{ for } k=2,3 \\
\alpha &= -2 \\
\gamma^{(0)} &= 1/2, \quad \gamma^{(1)} = 1/4.
\end{align*}

In [46]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
from scipy.special import logsumexp
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

import pyblp
pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

### 1. 
Draw the exogenous product characteristic $x_{jt}$ for $T=600$ geographically defined markets (e.g., cities). Assume each $x_{jt}$ is equal to the absolute value of an iid standard normal draw, as is each $w_{jt}$. Simulate demand and cost unobservables as well, specifying

\begin{equation*}
\left(
\begin{array}{c}
\xi_{jt} \\
\omega_{jt}
\end{array}
\right) \sim N\left( \left(
\begin{array}{c}
0 \\
0
\end{array}
\right), \left(
\begin{array}{cc}
1 & 0.25 \\
0.25 & 1
\end{array}
\right) \right) \quad \text{iid across } j,t.
\end{equation*}

In [47]:
np.random.seed(1995)

# Model parameters
T, J = 600, 4
alpha, beta1 = -2, 1
beta2, beta3 = 4, 4  
sigma_satellite, sigma_wired = 1, 1
gamma0, gamma1 = 0.5, 0.25

# Product data structure
data = [{'market_ids': t, 'firm_id': j+1, 'product_ids': j} for t in range(T) for j in range(J)]
product_data = pd.DataFrame(data)

# Exogenous variables: x_jt and w_jt as absolute values of iid standard normal draws
product_data['x'] = np.abs(np.random.normal(0, 1, len(product_data)))
product_data['w'] = np.abs(np.random.normal(0, 1, len(product_data)))

# Indicators
product_data['satellite'] = product_data['firm_id'].isin([1, 2]).astype(int)
product_data['wired'] = product_data['firm_id'].isin([3, 4]).astype(int)

# Unobservables: ξ_jt and ω_jt with covariance matrix [[1, 0.25], [0.25, 1]]
cov_matrix = np.array([[1, 0.25], [0.25, 1]])
A = np.linalg.cholesky(cov_matrix)
z = np.random.normal(0, 1, (len(product_data), 2))
unobs = z @ A.T
product_data['xi'] = unobs[:, 0]  # demand unobservable
product_data['omega'] = unobs[:, 1]  # cost unobservable

print("Question 1 completed:")
print(f"Generated {len(product_data)} observations across {T} markets")
print(f'x range: {product_data['x'].min():.3f} to {product_data['x'].max():.3f}')
print(f"w range: {product_data["w"].min():.3f} to {product_data["w"].max():.3f}")
print(f"ξ-ω correlation: {product_data[['xi', 'omega']].corr().iloc[0,1]:.3f} (target: 0.25)")
print(f"Satellite products: {product_data["satellite"].sum()}, Wired products: {product_data["wired"].sum()}")

Question 1 completed:
Generated 2400 observations across 600 markets
x range: 0.001 to 3.534
w range: 0.000 to 3.621
ξ-ω correlation: 0.240 (target: 0.25)
Satellite products: 1200, Wired products: 1200


### 2. Solve for the equilibrium prices for each good in each market.

**(a)** Start by writing a procedure to approximate the derivatives of market shares with respect to prices (taking prices, shares, x, and demand parameters as inputs). The key steps are:

(i) For each $jt$, write the choice probability for good $j$, $s_{jt}$, as a weighted average (integral) of the (multinomial logit) choice probabilities conditional on the value of each consumer's random coefficients;

The market share for good $j$ in market $t$, $s_{jt}$, is the probability that a consumer chooses good $j$:

$$s_{jt} = \int P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)}) f(\beta_i^{(2)}, \beta_i^{(3)}) d\beta_i^{(2)} d\beta_i^{(3)}$$

where $P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})$ is the multinomial logit choice probability conditional on the random coefficients.

Given the random coefficients $\beta_i^{(2)}$ and $\beta_i^{(3)}$ (with means $\beta^{(2)} = 4$, $\beta^{(3)} = 4$ and variances $\sigma_2^2 = 1$, $\sigma_3^2 = 1$), the conditional utility becomes:

$$u_{ijt} = \beta^{(1)} x_{jt} + \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt}$$

Since $\epsilon_{ijt}$ are i.i.d. Type-1 extreme value, the conditional choice probability follows the multinomial logit form:

$$P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)}) = \frac{\exp(\delta_{jt} + \mu_{jt}^i)}{\sum_{k=1}^J \exp(\delta_{kt} + \mu_{kt}^i) + 1}$$

where:
- $\delta_{jt} = \beta^{(1)} x_{jt} + \alpha p_{jt} + \xi_{jt}$ (mean utility component)
- $\mu_{jt}^i = \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt}$ (random utility component)

**Final Expression:**

$$s_{jt} = \int \frac{\exp(\delta_{jt} + \beta_i^{(2)} satellite_{jt} + \beta_i^{(3)} wired_{jt})}{\sum_{k=1}^J \exp(\delta_{kt} + \beta_i^{(2)} satellite_{kt} + \beta_i^{(3)} wired_{kt}) + 1} \phi(\beta_i^{(2)}, \beta_i^{(3)}) d\beta_i^{(2)} d\beta_i^{(3)}$$

where $\phi(\cdot, \cdot)$ is the bivariate normal density with mean $(\beta^{(2)}, \beta^{(3)}) = (4, 4)$ and covariance matrix $\text{diag}(1, 1)$.

This integral is approximated in the code using Monte Carlo simulation with draws from the normal distribution of $(\beta_i^{(2)}, \beta_i^{(3)})$.

(ii) Anticipating differentiation under the integral sign, derive the analytical expression for the derivative of the integrand with respect to each $p_{kt}$.

The integrand is the conditional choice probability $P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})$, which depends on prices through the mean utility component $\delta_{jt} = \beta^{(1)} x_{jt} + \alpha p_{jt} + \xi_{jt}$.

Since $p_{kt}$ appears in $\delta_{kt}$, the derivative with respect to $p_{kt}$ affects the choice probability.

For the multinomial logit model, the derivative of the choice probability with respect to a price is:

$$\frac{\partial P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})}{\partial p_{kt}} = \alpha P(j|\beta_i) \left( I_{jk} - P(k|\beta_i) \right)$$

where $I_{jk}$ is the indicator function equal to 1 if $j = k$.

Therefore, the derivative of the integrand (conditional choice probability) with respect to $p_{kt}$ is:

$$\frac{\partial}{\partial p_{kt}} \left[ \frac{\exp(\delta_{jt} + \mu_{jt}^i)}{\sum_{m=1}^J \exp(\delta_{mt} + \mu_{mt}^i) + 1} \right] = \alpha \cdot \frac{\exp(\delta_{jt} + \mu_{jt}^i)}{\sum_{m=1}^J \exp(\delta_{mt} + \mu_{mt}^i) + 1} \left( I_{jk} - \frac{\exp(\delta_{kt} + \mu_{kt}^i)}{\sum_{m=1}^J \exp(\delta_{mt} + \mu_{mt}^i) + 1} \right)$$

3. Use the expression you obtained in (2) and simulation draws of the random coefficients to approximate the integral that corresponds to $\partial s_{jt}/\partial p_{kt}$ for each $j$ and $k$ (i.e., replace the integral with the mean over the values at each simulation draw). Recall the advice in the lecture regarding "jittering."

In [48]:
def compute_market_shares(prices, market_data, v_draws=None, n_draws=1000):
    """Compute market shares using simulation"""
    if v_draws is None:
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
    J = len(market_data)
    x = market_data['x'].values
    xi = market_data['xi'].values
    sat = market_data['satellite'].values
    wired = market_data['wired'].values
    utilities = (beta1 * x + xi + v_draws[:, 0:1] * sat + v_draws[:, 1:2] * wired + alpha * prices)
    utilities = np.column_stack([utilities, np.zeros(v_draws.shape[0])])
    exp_u = np.exp(utilities - np.max(utilities, axis=1, keepdims=True))
    choice_probs = exp_u / exp_u.sum(axis=1, keepdims=True)
    shares = np.mean(choice_probs[:, :J], axis=0)
    return shares

def approximate_share_derivatives(prices, market_data, v_draws):
    """Approximate share derivatives using simulation"""
    J = len(market_data)
    x = market_data['x'].values
    xi = market_data['xi'].values
    sat = market_data['satellite'].values
    wired = market_data['wired'].values
    utilities = (beta1 * x + xi + v_draws[:, 0:1] * sat + v_draws[:, 1:2] * wired + alpha * prices)
    utilities = np.column_stack([utilities, np.zeros(v_draws.shape[0])])
    exp_u = np.exp(utilities - np.max(utilities, axis=1, keepdims=True))
    choice_probs = exp_u / exp_u.sum(axis=1, keepdims=True)
    inside_shares_draws = choice_probs[:, :J]
    derivatives = np.zeros((J, J))
    for j in range(J):
        for k in range(J):
            indicator = float(j == k)
            deriv_draws = alpha * inside_shares_draws[:, j] * (indicator - inside_shares_draws[:, k])
            derivatives[j, k] = np.mean(deriv_draws)
    return derivatives

def market_shares_and_derivatives(prices, market_data, v_draws):
    """Compute both shares and derivatives"""
    shares = compute_market_shares(prices, market_data, v_draws)
    derivatives = approximate_share_derivatives(prices, market_data, v_draws)
    return shares, derivatives

The derivative $\partial s_{jt}/\partial p_{kt}$ is approximated using Monte Carlo simulation. For each simulation draw $r = 1, \dots, R$ of the random coefficients $(\beta_i^{(2)}, \beta_i^{(3)})$, compute the conditional choice probability $P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})$ and its derivative with respect to prices.

The derivative of the conditional choice probability follows from the multinomial logit formula:

$$\frac{\partial P(\text{choose } j | \beta_i^{(2)}, \beta_i^{(3)})}{\partial p_{kt}} = \alpha P(j|\beta_i) \left( \delta_{jk} - P(k|\beta_i) \right)$$

where $\delta_{jk} = 1$ if $j = k` and 0 otherwise.

Then, the market share derivative is approximated as:

$$\frac{\partial s_{jt}}{\partial p_{kt}} \approx \frac{1}{R} \sum_{r=1}^R \frac{\partial P(\text{choose } j | \beta_i^{(2,r)}, \beta_i^{(3,r)})}{\partial p_{kt}}$$

Regarding "jittering": When solving for equilibrium prices iteratively, redrawing simulation draws in each iteration introduces random noise that can prevent convergence. To avoid this, pre-draw a fixed set of simulation draws for each market and reuse them throughout the solution process.

In [49]:
# Pre-draw simulation draws (to avoid jittering)
n_draws = 1000
all_v_draws = [np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws) for _ in range(T)]

(iv) Experiment to see how many simulation draws you need to get precise approximations and check this again at the equilibrium shares and prices you obtained below.

In [50]:
# Marginal costs: ln(mc_jt) = γ₀ + w_jt γ₁ + ω_jt/8
product_data['mc'] = np.exp(gamma0 + gamma1 * product_data['w'] + product_data['omega'] / 8)

# Test on market 0
market_data = product_data[product_data['market_ids'] == 0]
prices_initial = market_data['mc'].values
draw_counts = [50, 100, 200, 500, 1000, 2000, 5000]
print("Testing derivative approximation convergence at initial prices:")
print("Draws\t| Derivative Std Dev\t| Time (s)")
print("-" * 40)
initial_stds = []
for n_draws in draw_counts:
    start_time = time.time()

    # Compute derivatives at initial prices (5 repetitions for stability)
    deriv_initial_list = []
    for rep in range(5):
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
        deriv = approximate_share_derivatives(prices_initial, market_data, v_draws)
        deriv_initial_list.append(deriv)

    deriv_initial_avg = np.mean(deriv_initial_list, axis=0)
    deriv_initial_std = np.std(deriv_initial_list, axis=0)
    initial_stds.append(deriv_initial_std.mean())

    computation_time = time.time() - start_time

    print(f"{n_draws:6d}\t| {deriv_initial_std.mean():.2e}\t\t| {computation_time:.2f}")

# Determine stabilization point for initial prices
initial_stds = np.array(initial_stds)
threshold = 0.001
stable_idx = None
for i in range(1, len(initial_stds)):
    if initial_stds[i] < threshold and abs(initial_stds[i] - initial_stds[i-1]) / initial_stds[i-1] < 0.5:
        stable_idx = i
        break

if stable_idx is not None:
    stable_draws = draw_counts[stable_idx]
    print(f"\nCONCLUSION: At initial prices = MC, derivatives stabilize with {stable_draws} simulation draws.")
else:
    print(f"\nCONCLUSION: At initial prices = MC, derivatives show decreasing variance, with best stability at {draw_counts[np.argmin(initial_stds)]} draws.")

Testing derivative approximation convergence at initial prices:
Draws	| Derivative Std Dev	| Time (s)
----------------------------------------
    50	| 8.37e-03		| 0.00
   100	| 5.49e-03		| 0.00
   200	| 1.65e-03		| 0.00
   500	| 2.37e-03		| 0.00
  1000	| 1.54e-03		| 0.00
  2000	| 7.31e-04		| 0.00
  5000	| 4.65e-04		| 0.01

CONCLUSION: At initial prices = MC, derivatives stabilize with 5000 simulation draws.


(b) The FOC for firm $j$'s profit maximization problem in market $t$ is

\begin{align}
(p_{jt} - mc_{jt}) \frac{\partial s_{jt}}{\partial p_{jt}} + s_{jt} &= 0 \notag \\
\implies p_{jt} - mc_{jt} &= -\left( \frac{\partial s_{jt}}{\partial p_{jt}} \right)^{-1} s_{jt}
\end{align}

In [51]:

print(f"MC range: {product_data['mc'].min():.3f} to {product_data['mc'].max():.3f}")
print(f"MC mean: {product_data['mc'].mean():.3f}, median: {product_data['mc'].median():.3f}")
print("FOC: (p_jt - mc_jt) * ∂s_jt/∂p_jt + s_jt = 0")
print("Rearranged: p_jt - mc_jt = - (∂s_jt/∂p_jt)⁻¹ * s_jt")

MC range: 1.135 to 4.652
MC mean: 2.058, median: 1.994
FOC: (p_jt - mc_jt) * ∂s_jt/∂p_jt + s_jt = 0
Rearranged: p_jt - mc_jt = - (∂s_jt/∂p_jt)⁻¹ * s_jt


(c) Substituting in your approximation of each $\partial s_{jt}/\partial p_{jt}$, solve the system of equations above ($J$ equations per market) for the equilibrium prices in each market.

**i.** First do this using Matlab's "fsolve" operator. Check the exit flag from fsolve to be sure whether you found a solution for each market.

In [52]:
def solve_prices_direct(market_data, mc_market, v_draws):
    """Solve for equilibrium prices using direct nonlinear solver with robust matrix inversion"""
    J = len(market_data)
    
    def foc_residual(prices):
        """FOC residuals: p - mc + (∂s/∂p)^{-1} s = 0"""
        # Ensure prices are positive
        prices = np.maximum(prices, 1e-6)
        # Compute shares and derivatives at current prices
        shares, derivatives = market_shares_and_derivatives(prices, market_data, v_draws)
        # Robust inversion of derivative matrix
        try:
            invD = np.linalg.inv(derivatives)
        except np.linalg.LinAlgError:
            invD = np.linalg.inv(derivatives + 1e-8 * np.eye(J))
        # FOC residuals: p - mc + inv(∂s/∂p) @ s
        residuals = prices - mc_market + invD @ shares
        return residuals
    # Initial guess: marginal costs
    p0 = mc_market.copy()
    # Solve using root finder (hybr method)
    sol = opt.root(foc_residual, p0, method='hybr', tol=1e-8)
    prices_sol = sol.x
    success = sol.success
    # Additional check: verify that residuals are small
    final_residuals = foc_residual(prices_sol)
    if np.max(np.abs(final_residuals)) > 1e-6:
        success = False
    return prices_sol, success

# Solve using direct method
equilibrium_prices_direct = []
success_flags_direct = []
for t in range(T):
    market_data = product_data[product_data['market_ids'] == t]
    mc_market = market_data['mc'].values
    v_draws = all_v_draws[t]
    prices_direct, success = solve_prices_direct(market_data, mc_market, v_draws)
    equilibrium_prices_direct.append(prices_direct)
    success_flags_direct.append(success)
equilibrium_prices_direct = np.array(equilibrium_prices_direct)
success_count = sum(success_flags_direct)
print("Question 2(c)i completed:")
print(f"Direct nonlinear solver (root): {success_count}/{T} markets solved successfully")
print(f"Success rate: {success_count/T:.1%}")
print(f"Price range: {equilibrium_prices_direct.min():.3f} to {equilibrium_prices_direct.max():.3f}")
print(f"Price mean: {equilibrium_prices_direct.mean():.3f}, std: {equilibrium_prices_direct.std():.3f}")

Question 2(c)i completed:
Direct nonlinear solver (root): 600/600 markets solved successfully
Success rate: 100.0%
Price range: 2.349 to 5.699
Price mean: 3.317, std: 0.435


ii. Do this again using the algorithm of Morrow and Skerlos (2011), discussed in section 3.6 of Conlon and Gortmaker (2019) (and in the pyBLP "problem simulation tutorial"). Use the numerical integration approach you used in step (a) to approximate the terms defined in equation (25) of Conlon and Gortmaker. If you get different results using this method, resolve this discrepancy either by correcting your code or explaining why your preferred method is the one to be trusted.

In [53]:
def solve_prices_morrow_skerlos(market_data, mc_market, v_draws, max_iter=100, tol=1e-6):
    """Morrow-Skerlos algorithm"""
    prices = mc_market.copy()
    for iteration in range(max_iter):
        shares, derivatives = market_shares_and_derivatives(prices, market_data, v_draws)
        # But for Gamma, need inside_shares_draws, so recompute as in simulation
        x = market_data['x'].values
        xi = market_data['xi'].values
        sat = market_data['satellite'].values
        wired = market_data['wired'].values
        utilities = (beta1 * x + xi + v_draws[:, 0:1] * sat + v_draws[:, 1:2] * wired + alpha * prices)
        utilities = np.column_stack([utilities, np.zeros(v_draws.shape[0])])
        exp_u = np.exp(utilities - np.max(utilities, axis=1, keepdims=True))
        choice_probs = exp_u / exp_u.sum(axis=1, keepdims=True)
        inside_shares_draws = choice_probs[:, :len(market_data)]
        Lambda = np.diag(alpha * shares)
        Gamma = alpha * (inside_shares_draws.T @ inside_shares_draws) / v_draws.shape[0]
        diff = prices - mc_market
        zeta = np.linalg.solve(Lambda, Gamma.T @ diff - shares)
        prices_new = mc_market + zeta
        foc_residual = Lambda @ (prices - mc_market - zeta)
        if np.max(np.abs(foc_residual)) < tol:
            break
        prices = 0.5 * prices + 0.5 * prices_new
    return prices, iteration + 1

# Solve using Morrow-Skerlos method
equilibrium_prices_ms = []
iterations_ms = []

for t in range(T):
    market_data = product_data[product_data['market_ids'] == t]
    mc_market = market_data['mc'].values
    v_draws = all_v_draws[t]

    prices_ms, iters = solve_prices_morrow_skerlos(market_data, mc_market, v_draws)
    equilibrium_prices_ms.append(prices_ms)
    iterations_ms.append(iters)

equilibrium_prices_ms = np.array(equilibrium_prices_ms)
print("Question 2(c)ii completed:")
print(f"Morrow-Skerlos method: {T} markets solved")
print(f"Average iterations: {np.mean(iterations_ms):.1f}")
print(f"Max iterations: {np.max(iterations_ms)}")
print(f"Price range: {equilibrium_prices_ms.min():.3f} to {equilibrium_prices_ms.max():.3f}")
print(f"Price mean: {equilibrium_prices_ms.mean():.3f}, std: {equilibrium_prices_ms.std():.3f}")

# Compare direct vs Morrow-Skerlos if direct succeeded for all
if len(equilibrium_prices_direct) == T:
    price_diff = np.abs(np.array(equilibrium_prices_direct) - equilibrium_prices_ms)
    print(f"Max price difference between methods: {price_diff.max():.2e}")
    print(f"Mean price difference: {price_diff.mean():.2e}")
else:
    print("Direct method failed for some markets, skipsigmang comparison.")
    print("Preferred method: Morrow-Skerlos, as it is more numerically stable.")

# Use Morrow-Skerlos prices
product_data['prices'] = equilibrium_prices_ms.flatten()


Question 2(c)ii completed:
Morrow-Skerlos method: 600 markets solved
Average iterations: 27.9
Max iterations: 36
Price range: 2.349 to 5.699
Price mean: 3.317, std: 0.435
Max price difference between methods: 4.78e-06
Mean price difference: 1.26e-06


In [54]:
# Compare derivative approximation convergence at initial vs equilibrium prices

# Test on market 0
market_data = product_data[product_data['market_ids'] == 0]
prices_initial = market_data['mc'].values 
prices_equilibrium = market_data['prices'].values

draw_counts = [50, 100, 200, 500, 1000, 2000, 5000]

print("Comparing derivative approximation convergence at initial vs equilibrium prices:")
print("Draws\t| Initial Price Std Dev\t| Equilibrium Price Std Dev\t| Ratio (Eq/Init)")
print("-" * 80)

initial_stds = []
eq_stds = []
for n_draws in draw_counts:

    # Compute derivatives at initial prices
    deriv_initial_list = []
    for rep in range(5):
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
        deriv = approximate_share_derivatives(prices_initial, market_data, v_draws)
        deriv_initial_list.append(deriv)

    deriv_initial_std = np.std(deriv_initial_list, axis=0)
    initial_stds.append(deriv_initial_std.mean())

    # Compute derivatives at equilibrium prices
    deriv_eq_list = []
    for rep in range(5):
        v_draws = np.random.multivariate_normal([beta2, beta3], np.diag([sigma_satellite, sigma_wired]), size=n_draws)
        deriv = approximate_share_derivatives(prices_equilibrium, market_data, v_draws)
        deriv_eq_list.append(deriv)

    deriv_eq_std = np.std(deriv_eq_list, axis=0)
    eq_stds.append(deriv_eq_std.mean())

    ratio = deriv_eq_std.mean() / deriv_initial_std.mean() if deriv_initial_std.mean() > 0 else float('inf')

    print(f"{n_draws:6d}\t| {deriv_initial_std.mean():.2e}\t\t| {deriv_eq_std.mean():.2e}\t\t\t| {ratio:.2f}")


print(f"Average ratio of std dev (equilibrium/initial): {np.mean([eq_stds[i]/initial_stds[i] for i in range(len(initial_stds)) if initial_stds[i] > 0]):.2f}")

avg_ratio = np.mean([eq_stds[i]/initial_stds[i] for i in range(len(initial_stds)) if initial_stds[i] > 0])
reduction_pct = (1 - avg_ratio) * 100
print(f"\nCONCLUSION: Derivatives at equilibrium prices are, on average, {reduction_pct:.1f}% less variable than at initial prices.")

Comparing derivative approximation convergence at initial vs equilibrium prices:
Draws	| Initial Price Std Dev	| Equilibrium Price Std Dev	| Ratio (Eq/Init)
--------------------------------------------------------------------------------
    50	| 7.03e-03		| 4.37e-03			| 0.62
   100	| 4.91e-03		| 2.80e-03			| 0.57
   200	| 4.26e-03		| 1.44e-03			| 0.34
   500	| 1.40e-03		| 1.23e-03			| 0.88
  1000	| 1.07e-03		| 1.03e-03			| 0.96
  2000	| 8.24e-04		| 8.27e-04			| 1.00
  5000	| 5.82e-04		| 3.76e-04			| 0.65
Average ratio of std dev (equilibrium/initial): 0.72

CONCLUSION: Derivatives at equilibrium prices are, on average, 28.3% less variable than at initial prices.


### 3. 
Calculate "observed" market shares for your fake data set using your parameters, your draws of $x$, $w$, $\xi$, $\omega$, and your equilibrium prices.

In [55]:
observed_shares = []
for t in range(T):
    market_data = product_data[product_data['market_ids'] == t]
    prices_market = market_data['prices'].values
    shares_market = compute_market_shares(prices_market, market_data, n_draws=10000)
    observed_shares.extend(shares_market)

product_data['shares'] = observed_shares

print(f"Share range: {product_data['shares'].min():.3f} to {product_data['shares'].max():.3f}")
print(f"Share mean: {product_data['shares'].mean():.3f}, std: {product_data['shares'].std():.3f}")

# Validation: Check market share sums
market_share_sums = product_data.groupby('market_ids')['shares'].sum()
print(f"Market share sums (should be < 1):")
print(f"Average: {market_share_sums.mean():.3f}")
print(f"Min: {market_share_sums.min():.3f}, Max: {market_share_sums.max():.3f}")
print(f"Outside shares: {1 - market_share_sums.mean():.3f} (average)")

# Check by product type
satellite_shares = product_data[product_data['satellite'] == 1]['shares'].mean()
wired_shares = product_data[product_data['wired'] == 1]['shares'].mean()
print(f"Average satellite product share: {satellite_shares:.3f}")
print(f"Average wired product share: {wired_shares:.3f}")

Share range: 0.000 to 0.729
Share mean: 0.136, std: 0.122
Market share sums (should be < 1):
Average: 0.543
Min: 0.304, Max: 0.750
Outside shares: 0.457 (average)
Average satellite product share: 0.135
Average wired product share: 0.136


### 4. 

Below you'll be using $x$ and $w$ as instruments in the demand estimation. Check whether these appear to be good instruments in your fake data using some regressions of prices and market shares on the exogenous variables (or some function of them; see the related discussion in the coding tips). If you believe the instruments are not providing enough variation, modify the parameter choices above until you are satisfied. Report your final choice of parameters and the results you rely on to conclude that the instruments seem good enough.

In [56]:

# Create quadratic and interaction columns first
product_data['x**2'] = product_data['x'] ** 2
product_data['w**2'] = product_data['w'] ** 2
product_data['x*w'] = product_data['x'] * product_data['w']

# sum over competing goods in market t
product_data['sum_x_competitors'] = product_data.groupby('market_ids')['x'].transform('sum') - product_data['x']
product_data['sum_w_competitors'] = product_data.groupby('market_ids')['w'].transform('sum') - product_data['w']

# index of the other good in the same nest
product_data['x_other_in_nest'] = product_data.groupby(['market_ids', 'satellite'])['x'].transform('sum') - product_data['x']
product_data['w_other_in_nest'] = product_data.groupby(['market_ids', 'satellite'])['w'].transform('sum') - product_data['w']

# Use satellite and wired dummies instead of constant
Z = product_data[['satellite', 'wired', 'x', 'w', 'x**2', 'w**2', 'x*w', 'sum_x_competitors', 'sum_w_competitors', 'x_other_in_nest', 'w_other_in_nest']]

# Regression 1: Prices on extended instruments (Relevance check)
price_model = sm.OLS(product_data['prices'], Z).fit()
print("Regression: Prices ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Relevance Check)")
print(f"R-squared: {price_model.rsquared:.3f}")
print(f"F-statistic: {price_model.fvalue:.2f} (p-value: {price_model.f_pvalue:.2e})")
print()

# Regression 2: Market shares on extended instruments
share_model = sm.OLS(product_data['shares'], Z).fit()
print("Regression: Shares ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Relevance Check)")
print(f"R-squared: {share_model.rsquared:.3f}")
print(f"F-statistic: {share_model.fvalue:.2f} (p-value: {share_model.f_pvalue:.2e})")
print()

# Regression 3: Demand unobservable ξ on instruments (Exclusion check)
xi_model = sm.OLS(product_data['xi'], Z).fit()
print("Regression: ξ ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Exclusion Check)")
print(f"R-squared: {xi_model.rsquared:.3f}")
print(f"F-statistic: {xi_model.fvalue:.2f} (p-value: {xi_model.f_pvalue:.2e})")
print()
# cost unobservable exclusion check
omega_model = sm.OLS(product_data['omega'], Z).fit()
print("Regression: ω ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Exclusion Check)")
print(f"R-squared: {omega_model.rsquared:.3f}")
print(f"F-statistic: {omega_model.fvalue:.2f} (p-value: {omega_model.f_pvalue:.2e})")

# Assess instrument validity
weak_instruments = (price_model.f_pvalue >= 0.01 and share_model.f_pvalue >= 0.01) or (price_model.rsquared < 0.05 and share_model.rsquared < 0.05)
excluded_instruments = xi_model.f_pvalue < 0.01 or omega_model.f_pvalue < 0.01  
print()
print("FINAL PARAMETER CHOICE:")
if weak_instruments or excluded_instruments:
    print("Parameters need adjustment - instruments are weak or invalid.")
else:
    print(f"Demand: α = {alpha}, β^(1) = {beta1}, β_i^(2) ~ N({beta2}, {sigma_satellite}²), β_i^(3) ~ N({beta3}, {sigma_wired}²)")
    print(f"Supply: γ^(0) = {gamma0}, γ^(1) = {gamma1}")
    print("These parameters generate data with valid instruments and are retained as final.")

Regression: Prices ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Relevance Check)
R-squared: 0.510
F-statistic: 249.06 (p-value: 0.00e+00)

Regression: Shares ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Relevance Check)
R-squared: 0.364
F-statistic: 136.77 (p-value: 2.25e-226)

Regression: ξ ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Exclusion Check)
R-squared: 0.003
F-statistic: 0.63 (p-value: 7.92e-01)

Regression: ω ~ satellite + wired + x + w + x² + w² + x*w + sum_x_competitors + sum_w_competitors + x_other_in_nest + w_other_in_nest (Exclusion Check)
R-squared: 0.003
F-statistic: 0.70 (p-value: 7.30e-01)

FINAL PARAMETER CHOICE:
Demand: α = -2, β^(1) = 1, β_i^(2) ~ N(4, 1²), β_i^(3) ~ N(4, 1²)
Supply: γ^(0) = 0.5, γ^(1) = 0.25
These parameters generate dat

In [57]:
product_data.to_csv('blp.csv', index=False)
print(product_data.head(8))

   market_ids  firm_id  product_ids      x      w  satellite  wired     xi  \
0           0        1            0  1.241  0.919          1      0 -0.629   
1           0        2            1  1.471  2.068          1      0  1.005   
2           0        3            2  2.101  0.009          0      1 -2.595   
3           0        4            3  1.465  2.114          0      1 -0.403   
4           1        1            0  0.818  1.106          1      0  0.846   
5           1        2            1  1.057  0.265          1      0 -1.792   
6           1        3            2  1.535  0.750          0      1  0.082   
7           1        4            3  0.614  0.576          0      1 -0.624   

   omega     mc  prices  shares   x**2       w**2    x*w  sum_x_competitors  \
0  0.676  2.258   3.465   0.052  1.539  8.452e-01  1.141              5.037   
1 -1.620  2.258   3.465   0.334  2.163  4.278e+00  3.042              4.807   
2  0.954  1.862   2.989   0.056  4.415  7.799e-05  0.019    

## 4 Estimate Some Mis-specified Models

### 5. Estimate the plain multinomial logit model of demand by OLS (ignoring the endogeneity of prices).

For the plain multinomial logit model, the utility is:

$$u_{ijt} = \beta^{(1)} x_{jt} + \beta^{(2)} satellite_{jt} + \beta^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt} + \epsilon_{ijt}$$

This implies the log-odds ratio:

$$\ln\left(\frac{s_{jt}}{s_{0t}}\right) = \delta_{jt} = \beta^{(1)} x_{jt} + \beta^{(2)} satellite_{jt} + \beta^{(3)} wired_{jt} + \alpha p_{jt} + \xi_{jt}$$

We can estimate this by OLS, regressing the logit-transformed shares on the observed product characteristics.

In [58]:
# Compute outside shares for each market
product_data['outside_share'] = 1 - product_data.groupby('market_ids')['shares'].transform('sum')

# Compute logit delta: ln(s_jt / s_0t)
product_data['logit_delta'] = np.log(product_data['shares'] / product_data['outside_share'])

# OLS using matrix algebra (no intercept)
y = product_data['logit_delta'].values
X = product_data[['prices', 'x', 'satellite', 'wired' ]].values

# Compute OLS estimates: beta_hat = (X^T X)^(-1) X^T y
beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

# Compute residuals and clustered standard errors
y_hat = X @ beta_hat
residuals = y - y_hat
n, k = X.shape

# Clustered covariance matrix by market
clusters = product_data['market_ids'].values
unique_clusters = np.unique(clusters)
V = np.zeros((k, k))
for c in unique_clusters:
    mask = clusters == c
    X_c = X[mask]
    e_c = residuals[mask]
    V += X_c.T @ np.outer(e_c, e_c) @ X_c
cov_matrix_ols = np.linalg.inv(X.T @ X) @ V @ np.linalg.inv(X.T @ X)
se_ols = np.sqrt(np.diag(cov_matrix_ols))

# t-statistics and p-values
t_stats = beta_hat / se_ols
p_values = 2 * (1 - stats.norm.cdf(np.abs(t_stats)))

print("OLS Regression: ln(s_jt/s_0t) ~ x + satellite + wired + prices (no intercept)")
print("-" * 70)
param_names = ['x', 'satellite', 'wired', 'prices']
for i, param in enumerate(param_names):
    print(f"{param:12s}: {beta_hat[i]:8.3f} (SE: {se_ols[i]:.3f}, t: {t_stats[i]:6.2f}, p: {p_values[i]:.3f})")

OLS Regression: ln(s_jt/s_0t) ~ x + satellite + wired + prices (no intercept)
----------------------------------------------------------------------
x           :   -1.248 (SE: 0.050, t: -24.91, p: 0.000)
satellite   :    0.855 (SE: 0.033, t:  25.94, p: 0.000)
wired       :    1.762 (SE: 0.160, t:  11.01, p: 0.000)
prices      :    1.795 (SE: 0.158, t:  11.35, p: 0.000)


In [59]:
product_data['demand_instruments0'] = product_data['prices']
ols_problem = pyblp.Problem(pyblp.Formulation('0 + prices + x + satellite + wired '), product_data)
ols_problem

Dimensions:
 T    N     K1    MD 
---  ----  ----  ----
600  2400   4     4  

Formulations:
     Column Indices:          0      1       2        3  
--------------------------  ------  ---  ---------  -----
X1: Linear Characteristics  prices   x   satellite  wired

In [60]:
ols_results = ols_problem.solve(method='1s')
ols_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +1.85E-24     0        +1.52E+03          +1.93E+03    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices          x        satellite      wired   
-----------  -----------  -----------  -----------
 -1.25E+00    +8.55E-01    +1.76E+00    +1.79E+00 
(+5.10E-02)  (+3.24E-02)  (+1.65E-01)  (+1.64E-01)

In [61]:
pd.DataFrame(index=ols_results.beta_labels, data={
    ("Estimates", "Manual OLS"): beta_hat,
    ("Estimates", "PyBLP"): ols_results.beta.flat,
    ("SEs", "Manual OLS"): se_ols,
    ("SEs", "PyBLP"): ols_results.beta_se.flat,
})

Estimates               SEs       
          Manual OLS  PyBLP Manual OLS  PyBLP
prices        -1.248 -1.248      0.050  0.051
x              0.855  0.855      0.033  0.032
satellite      1.762  1.762      0.160  0.165
wired          1.795  1.795      0.158  0.164

### 6. 
Re-estimate the multinomial logit model of demand by two-stage
least squares, instrumenting for prices with the exogenous demand shifters $%
x $ and excluded cost shifters w. Discuss how the results differ from those
obtained by OLS.

In [62]:
# First stage: 
Z = product_data[['satellite', 'wired', 'x', 'w', 'x**2', 'w**2', 'x*w', 'sum_x_competitors', 'sum_w_competitors']].values  

# First stage OLS:
sigma_hat = np.linalg.inv(Z.T @ Z) @ Z.T @ product_data['prices'].values
prices_hat = Z @ sigma_hat

# Second stage: Regress logit_delta on x + satellite + wired + predicted_prices
y = product_data['logit_delta'].values
X_hat = np.column_stack([
    prices_hat,  # Use predicted prices from first stage
    product_data['x'].values,
    product_data['satellite'].values,
    product_data['wired'].values
])

# 2SLS estimates: beta_hat_iv = (X_hat^T X_hat)^(-1) X_hat^T y
beta_hat_iv = np.linalg.inv(X_hat.T @ X_hat) @ X_hat.T @ y

# Compute 2SLS standard errors 
residuals_iv = y - X_hat @ beta_hat_iv

# HC0 covariance
V = X_hat.T @ np.diag(residuals_iv**2) @ X_hat
cov_matrix_iv = np.linalg.inv(X_hat.T @ X_hat) @ V @ np.linalg.inv(X_hat.T @ X_hat)
se_iv = np.sqrt(np.diag(cov_matrix_iv)) 
t_stats_iv = beta_hat_iv / se_iv
p_values_iv = 2 * (1 - stats.norm.cdf(np.abs(t_stats_iv)))

print("2SLS IV Regression: ln(s_jt/s_0t) ~ x + satellite + wired + prices_hat (no intercept)")
print("First stage instruments: x, w, x², w², x*w, sum_x_competitors, sum_w_competitors")
print("-" * 80)
param_names = ['prices', 'x', 'satellite', 'wired']
for i, param in enumerate(param_names):
    print(f"{param:12s}: {beta_hat_iv[i]:8.3f} (SE: {se_iv[i]:.3f}, t: {t_stats_iv[i]:6.2f}, p: {p_values_iv[i]:.3f})")

2SLS IV Regression: ln(s_jt/s_0t) ~ x + satellite + wired + prices_hat (no intercept)
First stage instruments: x, w, x², w², x*w, sum_x_competitors, sum_w_competitors
--------------------------------------------------------------------------------
prices      :   -1.939 (SE: 0.069, t: -28.22, p: 0.000)
x           :    0.923 (SE: 0.031, t:  30.01, p: 0.000)
satellite   :    3.996 (SE: 0.220, t:  18.15, p: 0.000)
wired       :    4.037 (SE: 0.220, t:  18.32, p: 0.000)


In [63]:
# Add demand instruments for PyBLP
product_data['demand_instruments0'] = product_data['w']
product_data['demand_instruments1'] = product_data['x**2']
product_data['demand_instruments2'] = product_data['w**2']
product_data['demand_instruments3'] = product_data['x*w']
product_data['demand_instruments4'] = product_data['sum_x_competitors']
product_data['demand_instruments5'] = product_data['sum_w_competitors']

iv_problem = pyblp.Problem(pyblp.Formulation('0 + prices + x + satellite + wired'), product_data)
iv_problem

Dimensions:
 T    N     K1    MD 
---  ----  ----  ----
600  2400   4     9  

Formulations:
     Column Indices:          0      1       2        3  
--------------------------  ------  ---  ---------  -----
X1: Linear Characteristics  prices   x   satellite  wired

In [64]:
iv_results = iv_problem.solve(method='1s')
iv_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +4.05E+00     0        +1.26E+03          +2.79E+03    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices          x        satellite      wired   
-----------  -----------  -----------  -----------
 -1.94E+00    +9.23E-01    +4.00E+00    +4.04E+00 
(+6.35E-02)  (+3.53E-02)  (+2.08E-01)  (+2.09E-01)

In [65]:
pd.DataFrame(index=iv_results.beta_labels, data={
    ("Estimates", "Manual IV"): beta_hat_iv,
    ("Estimates", "PyBLP IV"): iv_results.beta.flat,
    ("SEs", "Manual IV"): se_iv,
    ("SEs", "PyBLP IV"): iv_results.beta_se.flat
})

Estimates                SEs         
          Manual IV PyBLP IV Manual IV PyBLP IV
prices       -1.939   -1.939     0.069    0.064
x             0.923    0.923     0.031    0.035
satellite     3.996    3.996     0.220    0.208
wired         4.037    4.037     0.220    0.209

### 7. Nested Logit Model Estimation

Now estimate a nested logit model by two-stage least squares, treating "satellite" and "wired" as the two nests for the inside goods. You will probably want to review the discussion of the nested logit in Berry (1994). Note that Berry focuses on the special case in which all the "nesting parameters" are the same; you should allow a different nesting parameter for each nest.



In Berry’s notation, this means letting the parameter become g(j) , where g (j) indicates the group (satellite
or wired) to which each inside good j belongs.

In [66]:
# Prepare data for nested logit
product_data['nesting_ids'] = product_data['satellite'].map({1: 'satellite', 0: 'wired'})

# add the quality and cost indexes of the other good in the same nest as good j
product_data['x_other_in_nest'] = product_data.groupby(['market_ids', 'nesting_ids'])['x'].transform('sum') - product_data['x']
product_data['w_other_in_nest'] = product_data.groupby(['market_ids', 'nesting_ids'])['w'].transform('sum') - product_data['w']
product_data['demand_instruments5'] = product_data['x_other_in_nest']
product_data['demand_instruments6'] = product_data['w_other_in_nest']

In [67]:
# Compute ln_within_share
product_data["group_share"] = product_data.groupby(["market_ids", "nesting_ids"])["shares"].transform("sum")
product_data["ln_within_share"] = np.log(product_data["shares"] / product_data["group_share"])

# Create nest-specific ln_within_share
product_data["ln_within_share_sat"] = product_data["ln_within_share"] * product_data["satellite"]
product_data["ln_within_share_wired"] = product_data["ln_within_share"] * product_data["wired"]

# Define variables
exog_vars = ["x", "satellite", "wired"]
endog_vars = ["prices", "ln_within_share_sat", "ln_within_share_wired"]
instr_vars = ["w", "x**2", "w**2", "x*w", "sum_x_competitors", "sum_w_competitors", "x_other_in_nest", "w_other_in_nest"]
Z_vars = exog_vars + instr_vars

# First stage: Z = exog + instr
Z = product_data[Z_vars].values

# First stage OLS for each endog
n_endog = len(endog_vars)
sigma_hat = np.zeros((Z.shape[1], n_endog))
endog_hat = np.zeros((len(product_data), n_endog))
for i, var in enumerate(endog_vars):
    y_endog = product_data[var].values
    sigma = np.linalg.inv(Z.T @ Z) @ Z.T @ y_endog
    sigma_hat[:, i] = sigma
    endog_hat[:, i] = Z @ sigma

# Second stage: Regress logit_delta on exog + predicted_endog, reordered to match PyBLP
y = product_data["logit_delta"].values
X_hat = np.column_stack([
    endog_hat[:, 0],  # prices_hat
    product_data["x"].values,
    product_data["satellite"].values,
    product_data["wired"].values,
    endog_hat[:, 1],  # ln_within_share_sat_hat
    endog_hat[:, 2]   # ln_within_share_wired_hat
])

# 2SLS estimates
beta_hat_iv = np.linalg.inv(X_hat.T @ X_hat) @ X_hat.T @ y

# Compute robust standard errors (HC0)
residuals_iv = y - X_hat @ beta_hat_iv
V = X_hat.T @ np.diag(residuals_iv**2) @ X_hat
cov_matrix_iv = np.linalg.inv(X_hat.T @ X_hat) @ V @ np.linalg.inv(X_hat.T @ X_hat)
se_iv = np.sqrt(np.diag(cov_matrix_iv))
t_stats_iv = beta_hat_iv / se_iv
p_values_iv = 2 * (1 - stats.norm.cdf(np.abs(t_stats_iv)))

print("2SLS IV Regression: ln(s_jt/s_0t) ~ prices + x + satellite + wired + ln_within_share_sat + ln_within_share_wired (no intercept)")
print("First stage instruments: " + ", ".join(Z_vars))
print("-" * 120)
param_names = ["prices", "x", "satellite", "wired", "ln_within_share_sat", "ln_within_share_wired"]
for i, param in enumerate(param_names):
    print(f"{param:20s}: {beta_hat_iv[i]:8.3f} (SE: {se_iv[i]:.3f}, t: {t_stats_iv[i]:6.2f}, p: {p_values_iv[i]:.3f})")

2SLS IV Regression: ln(s_jt/s_0t) ~ prices + x + satellite + wired + ln_within_share_sat + ln_within_share_wired (no intercept)
First stage instruments: x, satellite, wired, w, x**2, w**2, x*w, sum_x_competitors, sum_w_competitors, x_other_in_nest, w_other_in_nest
------------------------------------------------------------------------------------------------------------------------
prices              :   -1.607 (SE: 0.108, t: -14.82, p: 0.000)
x                   :    0.803 (SE: 0.044, t:  18.33, p: 0.000)
satellite           :    3.102 (SE: 0.623, t:   4.98, p: 0.000)
wired               :    3.364 (SE: 0.595, t:   5.66, p: 0.000)
ln_within_share_sat :    0.104 (SE: 0.505, t:   0.21, p: 0.837)
ln_within_share_wired:    0.321 (SE: 0.512, t:   0.63, p: 0.532)


In [68]:
# Nested logit formulation
nl_problem = pyblp.Problem(pyblp.Formulation('0 + prices + x + satellite + wired'), product_data)

In [69]:
rho_initial = [0.7, 0.7]  # Initial values for rho_sat and rho_wired
nl_results = nl_problem.solve(rho=rho_initial, method='1s')
nl_results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 1    +1.84E+00    +3.49E-09       +2.99E+00        +3.05E+02        0        +8.90E+02          +4.35E+04    

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:04       Yes          5             9     

Rho Estimates (Robust SEs in Parentheses):
 satellite      wired   
-----------  -----------
 +1.08E-01    +3.17E-01 
(+4.63E-01)  (+4.84E-01)

Beta Estimates (Robust SEs in Parentheses):
  prices          x        satellite      wired   
-----------  -----------  -----------  -----------
 -1.61E+00    +8

In [70]:
# Compare manual nested logit estimates with PyBLP nested logit estimates for beta
nested_beta_comparison = pd.DataFrame(index=nl_results.beta_labels, data={
    ("Estimates", "Manual Nested"): beta_hat_iv[:4],  # prices, x, satellite, wired
    ("Estimates", "PyBLP Nested"): nl_results.beta.flat,
    ("SEs", "Manual Nested"): se_iv[:4],
    ("SEs", "PyBLP Nested"): nl_results.beta_se.flat
})

print("Beta Comparison for Nested Logit:")
print(nested_beta_comparison)

# Compare rho estimates
nested_rho_comparison = pd.DataFrame(index=nl_results.rho_labels, data={
    ("Estimates", "Manual Nested"): beta_hat_iv[4:],  # rho_sat, rho_wired
    ("Estimates", "PyBLP Nested"): nl_results.rho.flat,
    ("SEs", "Manual Nested"): se_iv[4:],
    ("SEs", "PyBLP Nested"): nl_results.rho_se.flat
})

print("\nRho Comparison for Nested Logit:")
print(nested_rho_comparison)

Beta Comparison for Nested Logit:
              Estimates                        SEs             
          Manual Nested PyBLP Nested Manual Nested PyBLP Nested
prices           -1.607       -1.608         0.108        0.101
x                 0.803        0.803         0.044        0.042
satellite         3.102        3.108         0.623        0.637
wired             3.364        3.362         0.595        0.488

Rho Comparison for Nested Logit:
              Estimates                        SEs             
          Manual Nested PyBLP Nested Manual Nested PyBLP Nested
satellite         0.104        0.108         0.505        0.463
wired             0.321        0.317         0.512        0.484


### 8.
Using the nested logit results, provide a table comparing the estimated own-price elasticities to the true own-price elasticities.The procedure you developed above for approximating derivatives cannot be used for your estimates based
on the nested logit model. But because we have analytic expressions for market shares in the nested logit model,
you could either differentiate these or use “finite difference” approximation of derivatives.

In [71]:
# Compute average elasticities per product over all markets
rho_sat = beta_hat_iv[4]  # Extract scalar
rho_wired = beta_hat_iv[5]

estimated_elasticities = []
true_elasticities = []

for t in range(T):
    market_data = product_data[product_data['market_ids'] == t]
    shares_market = market_data['shares'].values
    prices_market = market_data['prices'].values
    for j in range(J):
        s_j = shares_market[j]
        p_j = prices_market[j]
        satellite = market_data.iloc[j]['satellite']
        wired = market_data.iloc[j]['wired']
        
        # True elasticity (RC logit approximation)
        elasticity_true = alpha * p_j * (1 - s_j)
        true_elasticities.append(elasticity_true)
        
        # Estimated elasticity (nested logit)
        if satellite == 1:
            rho_g = rho_sat
            s_j_g = market_data[market_data['satellite'] == 1]['shares'].sum()
        else:
            rho_g = rho_wired
            s_j_g = market_data[market_data['wired'] == 1]['shares'].sum()
        elasticity_est = alpha * p_j * (1 - s_j - rho_g * s_j_g * (1 - s_j / s_j_g))
        estimated_elasticities.append(elasticity_est)

product_data['true_elasticity'] = true_elasticities
product_data['estimated_elasticity'] = estimated_elasticities

# Table of average elasticities per product
avg_elasticities = product_data.groupby('product_ids')[['true_elasticity', 'estimated_elasticity']].mean()

# Add product labels
product_labels = ['Satellite 1', 'Satellite 2', 'Wired 1', 'Wired 2']
avg_elasticities.index = product_labels

print('Average Elasticities per Product:')
print(avg_elasticities)

Average Elasticities per Product:
             true_elasticity  estimated_elasticity
Satellite 1           -5.822                -5.722
Satellite 2           -5.676                -5.583
Wired 1               -5.732                -5.445
Wired 2               -5.803                -5.493


Provide two additional tables showing the true
matrix of diversion ratios and the diversion ratios implied by your estimates.

In [72]:
# Compute diversion ratios

# Function to compute nested logit shares
def compute_nested_logit_shares(prices, market_data, rho_sat, rho_wired):
    """Compute nested logit shares"""
    J = len(market_data)
    x = market_data['x'].values
    xi = market_data['xi'].values
    sat = market_data['satellite'].values
    wired = market_data['wired'].values
    
    # Mean utility
    delta = beta1 * x + xi + alpha * prices
    
    # Group shares
    sat_mask = sat == 1
    wired_mask = wired == 1
    
    # Within-group shares
    exp_delta_sat = np.exp(delta[sat_mask] / (1 - rho_sat))
    exp_delta_wired = np.exp(delta[wired_mask] / (1 - rho_wired))
    
    s_within_sat = exp_delta_sat / exp_delta_sat.sum()
    s_within_wired = exp_delta_wired / exp_delta_wired.sum()
    
    # Group shares
    denom_sat = (exp_delta_sat.sum())**(1 - rho_sat) + 1
    denom_wired = (exp_delta_wired.sum())**(1 - rho_wired) + 1
    
    s_group_sat = (exp_delta_sat.sum())**(1 - rho_sat) / denom_sat
    s_group_wired = (exp_delta_wired.sum())**(1 - rho_wired) / denom_wired
    
    # Individual shares
    shares = np.zeros(J)
    shares[sat_mask] = s_within_sat * s_group_sat
    shares[wired_mask] = s_within_wired * s_group_wired
    
    return shares

# Compute true diversion ratios (average over markets)
true_diversion_matrices = []
for t in range(T):
    market_data = product_data[product_data['market_ids'] == t]
    prices_market = market_data['prices'].values
    derivatives = approximate_share_derivatives(prices_market, market_data, all_v_draws[t])
    diversion = np.zeros((J, J+1))  # +1 for outside
    for j in range(J):
        for k in range(J):
            if j != k:
                diversion[j, k] = - derivatives[k, j] / derivatives[j, j] if derivatives[j, j] != 0 else 0
        # Outside diversion
        diversion[j, J] = 1 - np.sum(diversion[j, :J])
    true_diversion_matrices.append(diversion)

true_avg_diversion = np.mean(true_diversion_matrices, axis=0)

# Compute estimated diversion ratios analytically for nested logit, Conlon and Mortimer (2018)
estimated_diversion_matrices = []
for t in range(T):
    market_data = product_data[product_data['market_ids'] == t]
    shares_market = market_data['shares'].values
    sat_mask = market_data['satellite'].values == 1
    wired_mask = market_data['wired'].values == 1
    diversion_est = np.zeros((J, J+1))
    for j in range(J):
        if sat_mask[j]:
            rho_g = rho_sat
            nest_mask = sat_mask
        else:
            rho_g = rho_wired
            nest_mask = wired_mask
        s_g = shares_market[nest_mask].sum()
        s_j_g = shares_market[j] / s_g
        Z = rho_g + (1 - rho_g) * s_g
        denom = 1 / Z - s_j_g
        for k in range(J):
            if nest_mask[k]:
                s_k_g = shares_market[k] / s_g
                diversion_est[j, k] = s_k_g / denom if denom != 0 else 0
            else:
                diversion_est[j, k] = 0
        diversion_est[j, J] = 1 - np.sum(diversion_est[j, :J])
    estimated_diversion_matrices.append(diversion_est)

estimated_avg_diversion = np.mean(estimated_diversion_matrices, axis=0)

# Print tables
product_labels = ['Satellite 1', 'Satellite 2', 'Wired 1', 'Wired 2', 'Outside']

print("True Diversion Ratios Matrix (Averaged over Markets):")
true_df = pd.DataFrame(true_avg_diversion, index=product_labels[:4], columns=product_labels)
print(true_df)
print()

print("Estimated Diversion Ratios Matrix (Averaged over Markets):")
est_df = pd.DataFrame(estimated_avg_diversion, index=product_labels[:4], columns=product_labels)
print(est_df)

True Diversion Ratios Matrix (Averaged over Markets):
             Satellite 1  Satellite 2  Wired 1  Wired 2  Outside
Satellite 1        0.000        0.224    0.130    0.121    0.525
Satellite 2        0.214        0.000    0.131    0.123    0.532
Wired 1            0.124        0.130    0.000    0.214    0.533
Wired 2            0.122        0.129    0.225    0.000    0.523

Estimated Diversion Ratios Matrix (Averaged over Markets):
             Satellite 1  Satellite 2  Wired 1  Wired 2  Outside
Satellite 1        0.248        0.201    0.000    0.000    0.551
Satellite 2        0.192        0.265    0.000    0.000    0.544
Wired 1            0.000        0.000    0.448    0.301    0.251
Wired 2            0.000        0.000    0.313    0.413    0.274


## 5 Estimate the Correctly Specified Model

Use the pyBLP software to estimate the correctly specified model. Allow pyBLP to construct
approximations to the optimal instruments, using the exogenous demand shifters and exogenous
cost shifters.6
9. Report a table with the estimates of the demand parameters and standard errors. Do
this twice: once when you estimate demand alone, then again when you estimate jointly
with supply.
10. Using your preferred estimates from the prior step (explain your preference), provide
a table comparing the estimated own-price elasticities to the true own-price elasticities.
Provide two additional tables showing the true matrix of diversion ratios and the diversion
ratios implied by your estimates.

In [73]:
product_data = product_data.drop(columns=["nesting_ids", "demand_instruments1", "demand_instruments2", "demand_instruments3", "demand_instruments4", "demand_instruments5", "demand_instruments6"])

In [74]:
X1_formulation = pyblp.Formulation('0 + prices + x + satellite + wired')
X2_formulation = pyblp.Formulation('0 + satellite + wired')
product_formulations = (X1_formulation, X2_formulation)
product_formulations

(prices + x + satellite + wired, satellite + wired)

In [75]:
integration = pyblp.Integration('monte_carlo', size=1000, specification_options={'seed': 1995})
integration

Configured to construct nodes and weights with Monte Carlo simulation with options {seed: 1995}.

In [76]:
problem1 = pyblp.Problem(product_formulations, product_data, integration=mc_integration)
problem1

Dimensions:
 T    N      I      K1    K2    MD 
---  ----  ------  ----  ----  ----
600  2400  600000   4     2     4  

Formulations:
       Column Indices:             0        1        2        3  
-----------------------------  ---------  -----  ---------  -----
 X1: Linear Characteristics     prices      x    satellite  wired
X2: Nonlinear Characteristics  satellite  wired                  

In [77]:
results1 = problem1.solve(sigma=np.eye(2))

The model may be under-identified. The total number of unfixed parameters is 6, which is more than the total number of moments, 4. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


In [78]:
optimal_iv_results1 = results1.compute_optimal_instruments()
optimal_problem1 = optimal_iv_results1.to_problem()

In [79]:
optimal_problem1.solve(sigma=np.eye(2), method='2s')

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +3.47E-22    +1.40E-10       +3.02E+01        +4.35E+01        0        +4.18E+03          +3.10E+03    

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:20       Yes          7            12          37723       118376   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
 Sigma:     satellite      wired   
---------  -----------  -----------
satellite   +1.30E+00              
           (+2.29E-01)           

In [80]:
for seed in range(3):
    initial_sigma = np.diag(np.random.default_rng(seed).uniform(0, 10, 2))
    seed_results = optimal_problem1.solve(sigma=initial_sigma)
    print(f"Seed {seed}:")
    print(f"  Initial Sigma:\n{initial_sigma}")
    print(f"  Estimated Sigma:\n{seed_results.sigma}")
    print()

Seed 0:
  Initial Sigma:
[[6.36961687 0.        ]
 [0.         2.69786714]]
  Estimated Sigma:
[[1.29528007 0.        ]
 [0.         1.02910005]]

Seed 1:
  Initial Sigma:
[[5.11821625 0.        ]
 [0.         9.50463696]]
  Estimated Sigma:
[[1.29528007 0.        ]
 [0.         1.02910005]]

Seed 2:
  Initial Sigma:
[[2.61612134 0.        ]
 [0.         2.98491143]]
  Estimated Sigma:
[[1.29528007 0.        ]
 [0.         1.02910005]]



\item To display the average prices, use the following (where \texttt{changed\_prices} is the output of \texttt{compute\_prices} as in Post-Estimation Tutorial of pyBLP).
    \begin{verbatim}
 T, J= 600, 4
 print(changed_prices.reshape((T, J)).mean(axis= 0))
    \end{verbatim}

    \item To display the average elasticities and diversion rations, use the following (where \texttt{elasticities}, for example, is the output of \texttt{compute\_elasticities} in Post-Estimation Tutorial of pyBLP).
    \begin{verbatim}
T, J= 600, 4
print(elasticities.reshape((T, J, J)).mean(axis= 0))
    \end{verbatim}
    (These resemble what one would write in Matlab, but there are subtle issues behind it, including row-major order (Python) vs column-major order (Matlab).)

    \item To apply 15\% cost reduction by the merged firms, use the following.
   
    \begin{verbatim}
merger_costs= costs.copy()
merger_costs[product_data.merger_ids== 1]= 0.85*merger_costs[product_data.merger_ids== 1]
    \end{verbatim}
    \noindent where \texttt{costs} and \texttt{merger\_ids} are as in Post-Estimation Tutorial of pyBLP. (Using \texttt{merger\_costs= costs} instead of using \texttt{copy} could lead to an unexpected behavior.)


\end{itemize}